<div style="text-align:center">
    <img src="files/monolearn-logo.png" height="150px">
    <h1>Python course</h1>
    <h3>Session 05: Web scrapping, Beautiful Soup, Http requests</h3>
    <h4><a href="https://amzenterprise.ir/">Ali Momenzadeh</a></h5>
</div>

### Http requests

In [ ]:
# pip install requests

In [ ]:
import requests

r = requests.get('https://www.python.org')

# print(r)
#print(r.content)

with open("files/5/python_index.txt", "w", encoding="utf-8") as file:
    # file.write(str(r.content))
    file.write(r.text)

In [ ]:
import requests

url = "https://reqbin.com/echo/post/form"
data = "key1=value1&key2=value2"
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

r = requests.post(url, data=data, headers=headers)

print(r.text)

In [ ]:
import requests

payload = {'key1': 'value1', 'key2': 'value2'}

r = requests.post("https://httpbin.org/post", data=payload)

print(r.text)

### Web scrapping

In [ ]:
# pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://divar.ir/s/tehran"

res = requests.get(url)

if res.status_code == 200:
    soup = BeautifulSoup(res.text, "html.parser")

    post_cards = soup.find_all("div", class_="kt-post-card__body")

    filtered_post_cards = []

    for post_card in post_cards:
        descs = post_card.findAll("div", class_="kt-post-card__description")

        if len(descs)>0:
            desc = descs.pop()
            if desc is not None:
                if desc.get_text() == "توافقی":
                    filtered_post_cards.append(post_card)

    for post_card in filtered_post_cards:
        print(post_card.find("h2", class_="kt-post-card__title").get_text())

else:
    print("Error")

In [ ]:
# pip install numpy
# pip install pandas

In [ ]:
import os
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

#PRJ_FOLDER_ADDRESS = "files/5"
PRJ_FOLDER_ADDRESS = os.path.join("files","5")
# PAGE_FOLDER_ADDRESS = "files/5/pages"
PAGE_FOLDER_ADDRESS = os.path.join(PRJ_FOLDER_ADDRESS, "pages")
DATASET_ADDRESS = os.path.join(PRJ_FOLDER_ADDRESS, "dataset.csv")
PRICE_PER_METER_ADDRESS = os.path.join(PRJ_FOLDER_ADDRESS, "price_per_meter.csv")

# [url, number of pages, district, district_number]
URLS = np.array([
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B3%D8%B9%D8%A7%D8%AF%D8%AA-%D8%A2%D8%A8%D8%A7%D8%AF-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=",70,"saadat_abad", 2],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B2%D8%B9%D9%81%D8%B1%D8%A7%D9%86%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 50, "zaferanie", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%BE%D9%88%D9%86%DA%A9-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 35, "poonak", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%88%D9%84%D9%86%D8%AC%DA%A9-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 38, "velenjak", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A8%D9%84%D9%88%D8%A7%D8%B1-%D9%81%D8%B1%D8%AF%D9%88%D8%B3-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 17, "bolovar_ferdous", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%86%DB%8C%D8%A7%D9%88%D8%B1%D8%A7%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 27, "niavaran", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%BE%D8%A7%D8%B3%D8%AF%D8%A7%D8%B1%D8%A7%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 8, "pasdaran", 4],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%81%D8%B1%D9%85%D8%A7%D9%86%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 17, "farmanie", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B3%D8%A7%D8%B2%D9%85%D8%A7%D9%86-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 8, "sazman_barname", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B4%D9%87%D8%B1%DA%A9-%D8%BA%D8%B1%D8%A8-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 27, "shahrak_gharb", 2],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AC%D9%86%D8%AA-%D8%A2%D8%A8%D8%A7%D8%AF-%D8%AC%D9%86%D9%88%D8%A8%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 8, "janatabad_jonoobi", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AC%D8%B1%D8%AF%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 20, "jordan", 6],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%87%D8%B1%D9%88%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 14, "heravi", 4],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AF%D8%B1%D9%88%D8%B3-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 6, "daroos", 3],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AC%D9%86%D8%AA-%D8%A2%D8%A8%D8%A7%D8%AF-%D9%85%D8%B1%DA%A9%D8%B2%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 8, "janatabad_markazi", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B4%D9%87%D8%B1-%D8%B2%DB%8C%D8%A8%D8%A7-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 6, "shahr_ziba", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%DA%86%DB%8C%D8%AA%DA%AF%D8%B1-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 3, "chitgar", 22],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%82%DB%8C%D8%B7%D8%B1%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 16, "gheytarie", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A7%D9%84%D9%87%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 21, "elahie", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B5%D8%A7%D8%AF%D9%82%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 2, "sadeghie", 2],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%81%D8%B1%D8%B4%D8%AA%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 30, "fereshte", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%85%D8%AD%D9%85%D9%88%D8%AF%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 11, "mahmoodie", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B3%D8%AA%D8%A7%D8%B1%D8%AE%D8%A7%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 2, "satarkhan", 2],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B4%D9%87%D8%B1%D8%A7%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 5, "shahran", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A7%D8%B2%DA%AF%D9%84-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 2, "ozgol", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%85%DB%8C%D8%B1%D8%AF%D8%A7%D9%85%D8%A7%D8%AF-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 18, "mirdamad", 3],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B8%D9%81%D8%B1-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 11, "zafar", 3],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%BE%DB%8C%D8%B1%D9%88%D8%B2%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "piroozi", 13],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A7%D8%AE%D8%AA%DB%8C%D8%A7%D8%B1%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 20, "ekhtiarie", 3],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AA%D9%87%D8%B1%D8%A7%D9%86%D9%BE%D8%A7%D8%B1%D8%B3-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "tehran_pars", 8],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A2%DB%8C%D8%AA-%D8%A7%D9%84%D9%84%D9%87-%DA%A9%D8%A7%D8%B4%D8%A7%D9%86%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 5, "aytl_kashani", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%85%D8%B1%D8%B2%D8%AF%D8%A7%D8%B1%D8%A7%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 9, "marzdaran", 2],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A7%D8%B3%D8%AA%D8%A7%D8%AF-%D9%85%D8%B9%DB%8C%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 8, "ostad_moein", 9],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B3%DB%8C%D8%AF%D8%AE%D9%86%D8%AF%D8%A7%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 3, "seyed_khandan", 7],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AC%DB%8C%D8%AD%D9%88%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 6, "jeyhoon", 10],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AC%D9%86%D8%AA-%D8%A2%D8%A8%D8%A7%D8%AF-%D8%B4%D9%85%D8%A7%D9%84%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 4, "janatabad_shomali", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A8%D8%A7%D8%BA-%D9%81%DB%8C%D8%B6-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 7, "bagh_feyz", 5],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%DA%A9%D8%A7%D9%85%D8%B1%D8%A7%D9%86%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 7, "kamranie", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AF%D9%88%D9%84%D8%AA-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 8, "dolat", 3],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B3%D9%84%D8%B3%D8%A8%DB%8C%D9%84-%D8%B4%D9%85%D8%A7%D9%84%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 4, "selsebil_shomali", 10],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%86%D8%A7%D8%B1%D9%85%DA%A9-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "narmak", 8],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%85%D8%AC%DB%8C%D8%AF%DB%8C%D9%87-%D8%AC%D9%86%D9%88%D8%A8%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "majidie_jonoobi", 8],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%85%D8%AC%DB%8C%D8%AF%DB%8C%D9%87-%D8%B4%D9%85%D8%A7%D9%84%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "majidie_shomali", 4],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%DA%AF%DB%8C%D8%B4%D8%A7-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "gisha", 2],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%DB%8C%D8%A7%D9%81%D8%AA-%D8%A2%D8%A8%D8%A7%D8%AF-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "yaft_abad", 18],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AF%D8%A7%D8%B1%D8%A2%D8%A8%D8%A7%D8%AF-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "darabad", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AD%DA%A9%DB%8C%D9%85%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "hakimie", 4],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%86%D8%A7%D8%B2%DB%8C-%D8%A2%D8%A8%D8%A7%D8%AF-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "nazi_abad", 16],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A7%D9%85%DB%8C%D8%B1%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "amirie", 11],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A8%D8%B1%DB%8C%D8%A7%D9%86%DA%A9-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 2, "beryanak", 10],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B2%D9%85%D8%B2%D9%85-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "zamzam", 17],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A7%D9%81%D8%B3%D8%B1%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "afsarie", 15],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A2%D9%87%D9%86%DA%AF-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "ahang", 14],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B5%D8%A7%D8%AD%D8%A8%D9%82%D8%B1%D8%A7%D9%86%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 2, "saheb_gharanie", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A2%D8%B0%D8%B1%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "azari", 17],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%DA%86%DB%8C%D8%B0%D8%B1-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "chizar", 1],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%82%D9%84%D9%87%DA%A9-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 2, "gholhak", 3],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%88%D9%86%DA%A9-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 4, "vanak", 3],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A8%D9%87%D8%B4%D8%AA%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "beheshti", 4],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B9%D8%A8%D8%A7%D8%B3-%D8%A2%D8%A8%D8%A7%D8%AF-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "abas_abad", 4],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%DB%8C%D9%88%D8%B3%D9%81-%D8%A2%D8%A8%D8%A7%D8%AF-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 9, "yoosef_abad", 6],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%A7%D9%85%DB%8C%D8%B1%D8%A2%D8%A8%D8%A7%D8%AF-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 8, "amir_abad", 6],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%85%D9%86%DB%8C%D8%B1%DB%8C%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "monirie", 11],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D9%81%D9%84%D8%A7%D8%AD-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "falah", 17],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AC%DB%8C-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "jey", 9],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%AE%D8%B2%D8%A7%D9%86%D9%87-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "khazane", 16],
    ["https://shabesh.com/search/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4/%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86/%D8%B7%D8%B1%D8%B4%D8%AA-%D8%AA%D9%87%D8%B1%D8%A7%D9%86?page=", 1, "tarasht", 2], ])


def SavePagesAsTxt():
    urls = URLS
    path = PAGE_FOLDER_ADDRESS

    for i in range(len(urls)):
        for j in range(int(urls[i][1])):
            url = urls[i][0]+str(j+1)
            r = requests.get(url)

            file_name = urls[i][2]+str(j+1)+".txt"
            # 5/pages/saadatabad2.txt
            full_path = os.path.join(path, file_name)

            with open(full_path, "w", encoding="utf-8") as file:
                file.write(r.text)

            print(file_name + " saved.")

def GetDataFromTxt():
    df = pd.DataFrame(columns=['Area','Age','Rooms','District','PricePerMeter','Price'])
    urls = URLS

    len1 = len(urls)
    for row in range(len1):# For each district
        len2 = int(urls[row][1]) # For each page
        for counts in range(len2):

            # Read the file
            file_name = urls[row][2]+str(counts+1)+".txt"
            file_name = os.path.join(PAGE_FOLDER_ADDRESS,file_name)
            f = open(file_name, "r",encoding='utf-8')
            FullPageCode=f.read()

            soup = BeautifulSoup(FullPageCode,'html.parser')
            f.close()

            all_houses = soup.find_all('div', class_='list_infoBox__iv8WI')

            for i in range(len(all_houses)):
                house_div = BeautifulSoup(str(all_houses[i]),'html.parser')
                cost = house_div.find('span' , class_='list_infoPrice___aJXK')
                atr = house_div.find_all('span' , class_='px-1') # house attributes

                if re.sub(r'\D' , '' , cost.text) =='' :
                    continue

                # remove incomplete atr
                if len(atr) < 3:
                    continue
                
                # 0=Area(Meter),1=Age,2=Rooms,3=District,4=PricePerMeter,5=Price
                temp_one_house=pd.array([None]*6)
                temp_one_house[3] = urls[row][3]
                temp_one_house[0] = int(re.sub(r'\D' , '' , atr[0].text))
                temp_one_house[1] = 1401 - int(re.sub(r'\D' , '' ,atr[2].text ))
                temp_one_house[2] = int(re.sub(r'\D' , '' ,atr[1].text ))
                temp_one_house[5] = int(re.sub(r'\D' , '' , cost.text))
                temp_one_house[4] = int(temp_one_house[5]/temp_one_house[0])

                df.loc[df.shape[0]] = temp_one_house
            print(row,"from ",len1,"  &  ",counts,"from ",len2)

    df.to_csv(DATASET_ADDRESS)


def CalculateAveragePricePerMeter():
    """
    Calculate average of Price per Meter in each district and replace the average instead of
    the price per meter in the dataFrame depending on the district, also remove outlier data,
    depending on Difference main price per meter of house and the average value
    """

    dataset = pd.read_csv(DATASET_ADDRESS)
    dataset = dataset.iloc[:, 1:]
    dataset['PricePerMeter'] = dataset['PricePerMeter'].astype(float)

    districts = list(map(int, [item[3] for item in URLS]))
    districts_max = max(districts)
    districts_max += 1

    price_per_meter_df = pd.DataFrame(columns=['sum','count','avarage'],data=[[0,0,0]]*districts_max,dtype='float')

    for row in range(len(dataset)) :
        # Rescale PricePerMeter by dividing to 1M
        dataset.at[row,'PricePerMeter'] = dataset.loc[row][4]/1000000

        price_per_meter_df.at[int(dataset.loc[row][3]),'sum'] += dataset.loc[row][4]
        price_per_meter_df.at[int(dataset.loc[row][3]),'count'] += 1

    for row in range(price_per_meter_df.shape[0]):
        if price_per_meter_df.at[row,'count'] !=0 :
            price_per_meter_df.at[row,'avarage'] = price_per_meter_df.at[row,'sum']/price_per_meter_df.at[row,'count']

    price_per_meter_df.to_csv(PRICE_PER_METER_ADDRESS)
    dataset.to_csv(DATASET_ADDRESS)


# Remove outlier data (price per meter and averge difference is bigger than 20 (20M))
def RemoveOutliers():
    dataset = pd.read_csv(DATASET_ADDRESS, index_col=[0])
    price_per_meter_df = pd.read_csv(PRICE_PER_METER_ADDRESS)

    for row in range(len(dataset)) :
        if abs(dataset.at[row,'PricePerMeter'] - price_per_meter_df.at[int(dataset.at[row,'District']),'avarage']) < 20 :
            dataset.at[row,'PricePerMeter'] = price_per_meter_df.at[int(dataset.at[row,'District']),'avarage']
        else:
            print("house "+str(row)+" deleted")
            dataset = dataset.drop(row)
    
    # Remove unnecessary added "index" column
    dataset.reset_index(inplace=True)
    dataset.drop(columns=["index"], inplace=True)

    dataset.to_csv(DATASET_ADDRESS)

In [ ]:
SavePagesAsTxt()

In [ ]:
GetDataFromTxt()

In [ ]:
CalculateAveragePricePerMeter()

In [ ]:
RemoveOutliers()